Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
import time

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (400000, 28, 28) (400000,)
Validation set (15000, 28, 28) (15000,)
Test set (18000, 28, 28) (18000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (400000, 784) (400000, 10)
Validation set (15000, 784) (15000, 10)
Test set (18000, 784) (18000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [5]:
# Build a graph like this: linear regression, followed by a RELU which is inturn 
# followed by another linear regression. Then softamax is executed.
# SGD is used.

batch_size = 128
RELU_size = 1024  # Number of RELU nodes

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(None, image_size * image_size), name="train_data")
  tf_train_labels = tf.placeholder(tf.float32, shape=(None, num_labels), name="train_label")
    
  # Variables for linear regression #1.
  w1 = tf.Variable(tf.truncated_normal([image_size * image_size, RELU_size]), name="weight_1")
  b1 = tf.Variable(tf.zeros([RELU_size]), name="bias_1")
  # Variables for linear regression #2.  
  w2 = tf.Variable(tf.truncated_normal([RELU_size, num_labels]), name="weight_2")
  b2 = tf.Variable(tf.zeros([num_labels]), name="bias_2")
  
  # linear regression #1 computation.
  with tf.name_scope("Logits_1"):
    logits_1 = tf.matmul(tf_train_dataset, w1) + b1
  # RELU next
  relu_out = tf.nn.relu(logits_1, name="RELU")                           
  # linear regression #2 computation.
  with tf.name_scope("Logits_2"):
    logits_2 = tf.matmul(relu_out, w2) + b2
                                    
                                    
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits_2, tf_train_labels, 
        name="softmax_and_cross_entropy") + 0.0015*(
        tf.nn.l2_loss(w1) + tf.nn.l2_loss(b1) + tf.nn.l2_loss(w2) + tf.nn.l2_loss(b2)))
        
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  prediction = tf.nn.softmax(logits_2)
#Enable TensorBoard
writer = tf.train.SummaryWriter('./RELU_SGD_graph', graph=graph)
writer.close()
print("Done graph construction.")

Done graph construction.


In [6]:
# Run the graph built above 
num_steps = 3001
start_time = time.time()
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      feed_dict = {tf_train_dataset : valid_dataset, tf_train_labels : valid_labels}
      valid_predictions = prediction.eval(feed_dict=feed_dict)
      print("Validation accuracy: %.1f%%" % accuracy(valid_predictions, valid_labels))
  print("\nDone with SGD loop.\n")
  feed_dict = {tf_train_dataset : test_dataset, tf_train_labels : test_labels}
  test_predictions = prediction.eval(feed_dict=feed_dict)
  #test_predictions = session.run([prediction], feed_dict=feed_dict)
  print("Test accuracy: %.1f%%" % accuracy(test_predictions, test_labels))
print("\nTime it takes to run the TensorFlow graph:", time.time()-start_time)

Initialized
Minibatch loss at step 0: 939.852356
Minibatch accuracy: 3.1%
Validation accuracy: 32.9%
Minibatch loss at step 500: 223.470215
Minibatch accuracy: 82.8%
Validation accuracy: 80.5%
Minibatch loss at step 1000: 104.860962
Minibatch accuracy: 79.7%
Validation accuracy: 83.1%
Minibatch loss at step 1500: 49.178547
Minibatch accuracy: 90.6%
Validation accuracy: 84.5%
Minibatch loss at step 2000: 23.416645
Minibatch accuracy: 85.2%
Validation accuracy: 85.8%
Minibatch loss at step 2500: 11.223280
Minibatch accuracy: 89.8%
Validation accuracy: 87.4%
Minibatch loss at step 3000: 5.579077
Minibatch accuracy: 85.2%
Validation accuracy: 87.4%

Done with SGD loop.

Test accuracy: 93.1%

Time it takes to run the TensorFlow graph: 29.199730396270752


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [7]:
# Build a graph like this: linear regression, followed by a RELU which is inturn 
# followed by another linear regression. Then softamax is executed.
# SGD is used.

batch_size = 128
RELU_size = 1024  # Number of RELU nodes

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(None, image_size * image_size), name="train_data")
  tf_train_labels = tf.placeholder(tf.float32, shape=(None, num_labels), name="train_label")
    
  # Variables for linear regression #1.
  w1 = tf.Variable(tf.truncated_normal([image_size * image_size, RELU_size]), name="weight_1")
  b1 = tf.Variable(tf.zeros([RELU_size]), name="bias_1")
  # Variables for linear regression #2.  
  w2 = tf.Variable(tf.truncated_normal([RELU_size, num_labels]), name="weight_2")
  b2 = tf.Variable(tf.zeros([num_labels]), name="bias_2")
  
  # linear regression #1 computation.
  with tf.name_scope("Logits_1"):
    logits_1 = tf.matmul(tf_train_dataset, w1) + b1
  # RELU next
  relu_out = tf.nn.relu(logits_1, name="RELU")                           
  # linear regression #2 computation.
  with tf.name_scope("Logits_2"):
    logits_2 = tf.matmul(relu_out, w2) + b2
                                    
                                    
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits_2, tf_train_labels, 
        name="softmax_and_cross_entropy")+ 0.0015*(
        tf.nn.l2_loss(w1) + tf.nn.l2_loss(b1) + tf.nn.l2_loss(w2) + tf.nn.l2_loss(b2)))
        
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  prediction = tf.nn.softmax(logits_2)
#Enable TensorBoard
writer = tf.train.SummaryWriter('./RELU_SGD_graph', graph=graph)
writer.close()
print("Done graph construction.")

Done graph construction.


In [8]:
# Run the graph built above 
num_steps = 15
start_time = time.time()
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, prediction], feed_dict=feed_dict)
    if (step % 1 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      feed_dict = {tf_train_dataset : valid_dataset, tf_train_labels : valid_labels}
      valid_predictions = prediction.eval(feed_dict=feed_dict)
      print("Validation accuracy: %.1f%%" % accuracy(valid_predictions, valid_labels))
  print("\nDone with SGD loop.\n")
  feed_dict = {tf_train_dataset : test_dataset, tf_train_labels : test_labels}
  test_predictions = prediction.eval(feed_dict=feed_dict)
  #test_predictions = session.run([prediction], feed_dict=feed_dict)
  print("Test accuracy: %.1f%%" % accuracy(test_predictions, test_labels))
print("\nTime it takes to run the TensorFlow graph:", time.time()-start_time)

Initialized
Minibatch loss at step 0: 824.173889
Minibatch accuracy: 12.5%
Validation accuracy: 34.1%
Minibatch loss at step 1: 1640.453125
Minibatch accuracy: 40.6%
Validation accuracy: 30.0%
Minibatch loss at step 2: 2246.655029
Minibatch accuracy: 24.2%
Validation accuracy: 32.8%
Minibatch loss at step 3: 1670.194458
Minibatch accuracy: 31.2%
Validation accuracy: 42.7%
Minibatch loss at step 4: 1062.812256
Minibatch accuracy: 43.8%
Validation accuracy: 52.7%
Minibatch loss at step 5: 685.905396
Minibatch accuracy: 57.8%
Validation accuracy: 59.5%
Minibatch loss at step 6: 658.508667
Minibatch accuracy: 60.9%
Validation accuracy: 73.1%
Minibatch loss at step 7: 604.178833
Minibatch accuracy: 75.8%
Validation accuracy: 73.3%
Minibatch loss at step 8: 598.682922
Minibatch accuracy: 67.2%
Validation accuracy: 73.4%
Minibatch loss at step 9: 541.622986
Minibatch accuracy: 71.1%
Validation accuracy: 76.9%
Minibatch loss at step 10: 532.075928
Minibatch accuracy: 79.7%
Validation accuracy:

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

With a few batches of training data, overfitting isn't obviously observed. In the above learning and testing, only in step 10 and 11, mini batch accuracy is higher than validation accuracy. In other steps, validation accuracy is higher than learning/training accuracy. This is not a strong evidence of overfitting. This happened with regularization removed. If there is regularization in cost, the result is similar.   

---

In [9]:
# Build a graph like this: linear regression with optional drop-out, followed by a RELU which is inturn 
# followed by another linear regression. Then softamax is executed.
# SGD is used.

batch_size = 128
RELU_size = 1024  # Number of RELU nodes

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(None, image_size * image_size), name="train_data")
  tf_train_labels = tf.placeholder(tf.float32, shape=(None, num_labels), name="train_label")
  keep_prob = tf.placeholder(tf.float32, name ="keep_prob")
                                    
  # Variables for linear regression #1.
  w1 = tf.Variable(tf.truncated_normal([image_size * image_size, RELU_size], stddev=1), name="weight_1")
  b1 = tf.Variable(tf.zeros([RELU_size]), name="bias_1")
  # Variables for linear regression #2.  
  w2 = tf.Variable(tf.truncated_normal([RELU_size, num_labels], stddev=1), name="weight_2")
  b2 = tf.Variable(tf.zeros([num_labels]), name="bias_2")
  
  # linear regression #1 computation.
  with tf.name_scope("Logits_1"):
    logits_1 = tf.nn.dropout(tf.matmul(tf_train_dataset, w1) + b1, keep_prob)
        
  # RELU next
  relu_out = tf.nn.relu(logits_1, name="RELU")                           
  # linear regression #2 computation.
  with tf.name_scope("Logits_2"):
    logits_2 = tf.matmul(relu_out, w2) + b2
                                    
                                    
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits_2, tf_train_labels, 
        name="softmax_and_cross_entropy")+ 0.0015*(
        tf.nn.l2_loss(w1) + tf.nn.l2_loss(b1) + tf.nn.l2_loss(w2) + tf.nn.l2_loss(b2)))
        
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  prediction = tf.nn.softmax(logits_2)
#Enable TensorBoard
writer = tf.train.SummaryWriter('./RELU_SGD_graph', graph=graph)
writer.close()
print("Done graph construction.")

Done graph construction.


In [10]:
# Run the graph built above 
num_steps = 3501
start_time = time.time()
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    in_training = True
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob:0.8}
    _, l, predictions = session.run(
      [optimizer, loss, prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      feed_dict = {tf_train_dataset : valid_dataset, tf_train_labels : valid_labels, keep_prob:1}
      valid_predictions = prediction.eval(feed_dict=feed_dict)
      print("Validation accuracy: %.1f%%" % accuracy(valid_predictions, valid_labels))
  print("\nDone with SGD loop.\n")
  feed_dict = {tf_train_dataset : test_dataset, tf_train_labels : test_labels, keep_prob:1}
  test_predictions = prediction.eval(feed_dict=feed_dict)
  #test_predictions = session.run([prediction], feed_dict=feed_dict)
  print("Test accuracy: %.1f%%" % accuracy(test_predictions, test_labels))
print("\nTime it takes to run the TensorFlow graph:", time.time()-start_time)

Initialized
Minibatch loss at step 0: 900.802734
Minibatch accuracy: 4.7%
Validation accuracy: 32.2%
Minibatch loss at step 500: 225.652100
Minibatch accuracy: 78.1%
Validation accuracy: 81.6%
Minibatch loss at step 1000: 105.286224
Minibatch accuracy: 80.5%
Validation accuracy: 82.9%
Minibatch loss at step 1500: 49.047386
Minibatch accuracy: 88.3%
Validation accuracy: 84.6%
Minibatch loss at step 2000: 23.440819
Minibatch accuracy: 82.8%
Validation accuracy: 84.9%
Minibatch loss at step 2500: 11.292606
Minibatch accuracy: 82.8%
Validation accuracy: 86.9%
Minibatch loss at step 3000: 5.598680
Minibatch accuracy: 84.4%
Validation accuracy: 86.8%
Minibatch loss at step 3500: 2.959666
Minibatch accuracy: 83.6%
Validation accuracy: 87.8%

Done with SGD loop.

Test accuracy: 93.8%

Time it takes to run the TensorFlow graph: 36.150553464889526


---
Problem 3 Comments
---------

With the above computation graph, the best results are as below:
RELU_size=1024, beta=0.0015, 3501 taining iterations, no drop_out, test accuracy: **93.7%**

If drop_out is set to 20%, and all other parameters are the same as the above, test accuracy: **93.4%**

If RELU_size is increased to 1536, and drop_out is kept at 20%, test accuracy: **93.7%**

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [41]:
import math
# Build a graph like this: linear regression with optional drop-out, followed by a RELU, 
# linear regression and RELU is repeated one more time, 
# and another final linear regression for output. Then softamax is executed.
# SGD is used to produce inputs.

batch_size = 128
l1_neros = 1024  # Number of RELU nodes
l2_neros = 512
l3_neros = 256
l4_neros =128

graph2 = tf.Graph()
with graph2.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(None, image_size * image_size), name="train_data")
  tf_train_labels = tf.placeholder(tf.float32, shape=(None, num_labels), name="train_label")
  keep_prob = tf.placeholder(tf.float32, name ="keep_prob")
                                    
  # Variables for linear regression #1.
  w1 = tf.Variable(tf.truncated_normal([image_size * image_size, l1_neros], stddev=1.414/image_size), name="weight_1")
  b1 = tf.Variable(tf.zeros([l1_neros]), name="bias_1")
  # Variables for linear regression #2.  
  w2 = tf.Variable(tf.truncated_normal([l1_neros, l2_neros], stddev=math.sqrt(2.0/l1_neros)), name="weight_2")
  b2 = tf.Variable(tf.zeros([l2_neros]), name="bias_2")
  # Variables for linear regression #3.  
  w3 = tf.Variable(tf.truncated_normal([l2_neros, l3_neros], stddev=math.sqrt(2.0/l2_neros)), name="weight_3")
  b3 = tf.Variable(tf.zeros([l3_neros]), name="bias_3")
  # Variables for linear regression #4.  
  w4 = tf.Variable(tf.truncated_normal([l3_neros, l4_neros], stddev=math.sqrt(2.0/l3_neros)), name="weight_4")
  b4 = tf.Variable(tf.zeros([l4_neros]), name="bias_4")
  # Variables for linear regression #5.  
  w5 = tf.Variable(tf.truncated_normal([l4_neros, num_labels], stddev=math.sqrt(2.0/l4_neros)), name="weight_5")
  b5 = tf.Variable(tf.zeros([num_labels]), name="bias_5")
  
    
  # linear regression #1 computation.
  with tf.name_scope("Logits_1"):
    logits_1 = tf.nn.dropout(tf.matmul(tf_train_dataset, w1) + b1, 1) #keep_prob)
  # RELU next
  relu_out_1 = tf.nn.relu(logits_1, name="RELU_1")                           

  # linear regression #2 computation.
  with tf.name_scope("Logits_2"):
    logits_2 = tf.nn.dropout(tf.matmul(relu_out_1, w2) + b2, 1) # keep_prob)
  # RELU next
  relu_out_2 = tf.nn.relu(logits_2, name="RELU_2")                           

  # linear regression #3 computation.
  with tf.name_scope("Logits_3"):
    logits_3 = tf.nn.dropout(tf.matmul(relu_out_2, w3) + b3, 1) #keep_prob)
  # RELU next
  relu_out_3 = tf.nn.relu(logits_3, name="RELU_3")                           

  # linear regression #4 computation.
  with tf.name_scope("Logits_4"):
    logits_4 = tf.nn.dropout(tf.matmul(relu_out_3, w4) + b4, 1) #keep_prob)
  # RELU next
  relu_out_4 = tf.nn.relu(logits_4, name="RELU_4")                           

  # linear regression #4 computation.
  with tf.name_scope("Logits_5"):
    logits = tf.matmul(relu_out_4, w5) + b5
    
                                    
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels, 
        name="softmax_and_cross_entropy")+ 0.00005*(
        tf.nn.l2_loss(w1) + tf.nn.l2_loss(b1) + tf.nn.l2_loss(w2) + tf.nn.l2_loss(
            b2)  + tf.nn.l2_loss(w3) + tf.nn.l2_loss(b3) + tf.nn.l2_loss(w4) + tf.nn.l2_loss(
            b4)  + tf.nn.l2_loss(w5) + tf.nn.l2_loss(b5)))
       
  
  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.5, global_step, 300, 0.95, staircase=True, name="Learn_rate_decay")
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  tf.scalar_summary(b'learning_rate', learning_rate, name="learning_rate_summary")
  merged_summaries = tf.merge_all_summaries()
  
  # Predictions for the training, validation, and test data.
  prediction = tf.nn.softmax(logits)
  prediction_accuracy = accuracy(prediction, tf_train_labels)
#Enable TensorBoard
writer = tf.train.SummaryWriter('./RELU_SGD_graph_2', graph=graph2)
print("Done graph2 construction.")

Done graph2 construction.


In [42]:
# Run the graph built above 
num_steps = 35501
start_time = time.time()
sess = tf.Session(graph=graph2)
with sess:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob:0.5}
    _, l, predictions = sess.run(
      [optimizer, loss, prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      feed_dict = {tf_train_dataset : valid_dataset, tf_train_labels : valid_labels, keep_prob:1}
      valid_predictions = prediction.eval(feed_dict=feed_dict)
      valid_accuracy = accuracy(valid_predictions, valid_labels)
      print("Validation accuracy: %.1f%%" % valid_accuracy)
      tf.scalar_summary(b'validation_accuracy', valid_accuracy, name="validation_accuracy_summary")
      summaries = merged_summaries.eval()
      writer.add_summary(summaries, global_step=step)
  print("\nDone with SGD loop.\n")
  feed_dict = {tf_train_dataset : test_dataset, tf_train_labels : test_labels, keep_prob:1}
  test_predictions = prediction.eval(feed_dict=feed_dict)
  print("Test accuracy: %.1f%%" % accuracy(test_predictions, test_labels))
writer.flush()
writer.close()
sess.close()
print("\nTime it takes to run the TensorFlow graph:", time.time()-start_time)

Initialized
Minibatch loss at step 0: 2.406985
Minibatch accuracy: 10.9%
Validation accuracy: 34.7%
Minibatch loss at step 500: 0.479293
Minibatch accuracy: 86.7%
Validation accuracy: 86.0%
Minibatch loss at step 1000: 0.588111
Minibatch accuracy: 84.4%
Validation accuracy: 87.4%
Minibatch loss at step 1500: 0.374860
Minibatch accuracy: 91.4%
Validation accuracy: 88.7%
Minibatch loss at step 2000: 0.430597
Minibatch accuracy: 87.5%
Validation accuracy: 89.1%
Minibatch loss at step 2500: 0.413264
Minibatch accuracy: 90.6%
Validation accuracy: 89.9%
Minibatch loss at step 3000: 0.381961
Minibatch accuracy: 90.6%
Validation accuracy: 89.8%
Minibatch loss at step 3500: 0.386371
Minibatch accuracy: 91.4%
Validation accuracy: 90.5%
Minibatch loss at step 4000: 0.395822
Minibatch accuracy: 92.2%
Validation accuracy: 90.7%
Minibatch loss at step 4500: 0.294786
Minibatch accuracy: 93.0%
Validation accuracy: 90.7%
Minibatch loss at step 5000: 0.239944
Minibatch accuracy: 95.3%
Validation accurac